# Mount google dirve

In [ ]:
#Mounting G-Drive and reading in libraries
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#import libraries
import numpy as np
import pandas as pd
import re
import string
import os
import math
import torch
import csv
import tensorflow as tf
from torch import tensor
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#set variables for folder names:
datain = 'here'
dataout = 'here' #make sure dir starts with /
clinical = 'here'

# Trasnformers similarity scores

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration, T5Model
T5_PATH = "t5-small" # "t5-small", "t5-base", "t5-large" [the best a GPU-computer can do, very slow], "t5-3b", "t5-11b"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

t5_tokenizer = T5Tokenizer.from_pretrained(T5_PATH)
#what is t5 tokenizer doing todo
t5_config = T5Config.from_pretrained(T5_PATH)
t5_mlm = T5ForConditionalGeneration.from_pretrained(T5_PATH, config=t5_config).to(DEVICE)

In [ ]:
type(t5_tokenizer) 

# Clean verbatim

In [ ]:
interjections = ['ach', 'duh', 'eee', 'ew', 'ha', 'hee', 'hm', 'huh', 'jeez',
                 'mmm', 'nah', 'okay', 'oof', 'oops', 'oh', 'phew', 
                 'um', 'uh', 'mm', 'mhm', 
                 'uh-oh',
                 'whoa', 'whoops', 'woo-hoo', 'woot', 'wow', 'yay', 'yeah', 'yep']

def clean_verbatim(content):
  for s in content.split(' '):
    #print(s)
    if s.lower().endswith('-') or '+' in s or s.lower() in interjections:
      content = content.replace(s, '')
  return content
# remove repetitions todo

st = "I'm si+ hm six-years go- uh is good Hmmmm so in ~A~B "
st = clean_verbatim(st)
st = " ".join(st.split())
print('st ', st)

st1 = 'I\'m six-years is good so in ~A~B   '

similarity_prob = "stsb sentence1: "+ st + " sentence2: "+ st1

print('flag1: ', similarity_prob) 

# use layers of LMs separately to measure embeddings ??                     
encoded = t5_tokenizer.encode(similarity_prob, return_tensors="pt").to(DEVICE)
print('encoded: ', encoded, len(encoded))
outputs2 = t5_mlm.generate(encoded)
print('outputs2: ', outputs2)
similarity_rating = t5_tokenizer.decode(outputs2[0], skip_special_tokens=True)
print("similarity1:",similarity_rating)

## clean remora aggregate sent deident

In [ ]:
df_sent = pd.read_csv(datain + 'sentence_clean.csv')
#df_sent.head(5)
taskin = ['AboutYourself','HowsItGoing','Hinting','Rorschach8','CookieTheft','TATPicture']
timepointin = ['BL']
speakerin = ['Subject']
df_sent = df_sent[df_sent['task'].isin(taskin)]
df_sent = df_sent[df_sent['timepoint'].isin(timepointin)]
df_sent = df_sent[df_sent['speaker'].isin(speakerin)]

df_sent = df_sent[df_sent['n_words'] > 3]

df_sent['t5small_similarity'] = ""

df_sent.head(5)

In [ ]:
def clean_verbatim(content):
  for s in content.split(' '):
    #print(s)
    if s.endswith('-') or s.lower() in interjections:
      content = content.replace(s, '')
      content = " ".join(content.split())
  return content

df_sent['content'].replace(r'=','', inplace=True)
df_sent['content'].replace(r'\'=', '', inplace=True)
df_sent['content'].replace(r'\^', '', inplace=True)
df_sent['content'].replace(r'\+', '', inplace=True)
df_sent['content'].replace(r'\#', value='', inplace=True, regex = True)

df_sent['content'].replace(r'\{[A-Za-z]+\}', value='', inplace=True, regex = True)
df_sent['content'].replace(r'\(\([A-Za-z\s]+\)\)', value='', inplace=True)

df_sent['content'] = df_sent['content'].apply(lambda x: clean_verbatim(x))


In [ ]:
df_turn['content'].replace('=','', inplace=True)
df_turn['content'].replace('\'=', '', inplace=True)
df_turn['content'].replace('\^', '', inplace=True)
df_turn['content'].replace(r'\{[A-Za-z]+\}', value='', inplace=True, regex = True)
df_turn['content'].replace('\(\(\)\)', value='', inplace=True)
df_turn['content'].replace(to_replace=r'\(\(', value='', inplace=True, regex = True)
df_turn['content'].replace(to_replace=r'\)\)', value='', inplace=True, regex = True)
df_turn['content'].replace(to_replace=r'(?<!\()\((?!\()', value='', inplace=True, regex = True)
df_turn['content'].replace(to_replace=r'\)\=', value='', inplace=True, regex = True)
df_turn['content'].replace(to_replace=r'\#', value='', inplace=True, regex = True)


# Run encoder-decoder T5 for adjacent sententence

## similarity: sententence

In [ ]:
df_sent = pd.read_csv(datain + 'sentence_clean.csv')
for i in df_sent.index:
  text = df_sent['content'][i]
  text = re.sub(r'\s([?.!,"](?:\s|$))', r'\1', text) #replace ' .' with '.'
  df_sent['content'][i] = text

df_sent.tail()

In [ ]:
buglst = {}
for i,r in df_sent.iterrows():
  if r['speaker'] != 'Interviewer':
    try:
      seq1 = r['content']
      #print('seq1: ', seq1)

      seq2 = df_sent['content'][i+1]
      #print('seq2: ', seq2)

      similarity_prob = "stsb sentence1: " + seq1 + " sentence2: " + seq2
      print('sentence pair: ', similarity_prob) 
                            
      encoded = t5_tokenizer.encode(similarity_prob, return_tensors="pt").to(DEVICE)
      outputs2 = t5_mlm.generate(encoded)
      similarity_rating = t5_tokenizer.decode(outputs2[0], skip_special_tokens=True)
      print("similarity: ", similarity_rating)

      df_sent['t5small_similarity'][i+1] = similarity_rating
    except KeyError:
      buglst[df_sent['Unnamed: 0'][i]] = df_sent['content'][i]
      continue


### min, max, std

In [ ]:
import pandas as pd
import numpy as np

t5 = pd.read_csv('sentence_clean.csv', index_col=0)
t5.head()

In [ ]:
t5['sentence_count'] = ''
buglst = {}
s = 0
for i in t5.index:
  try:
    if t5['task'][i] == t5['task'][i+1]:
      s += 1
    else: 
      t5['sentence_count'][i] = s + 1
      s = 0
  except:
    buglst[t5['Unnamed: 0.1'][i]] = t5['content'][i]

In [ ]:
t5 = t5[['uid', 'grid', 'timepoint', 'speaker', 'task', 't5small_similarity', 'sentence_count']]

In [ ]:
t5 = t5[t5.timepoint == 'BL']
t5 = t5[t5.task.isin(['AboutYourself', 'HowsItGoing'])]
t5 = t5.reset_index(drop=True)
len(t5.uid.unique())

In [ ]:
len(t5.uid.unique())

In [ ]:
t5['sentence_count'] = pd.to_numeric(t5['sentence_count'], errors='coerce')
t5['t5small_similarity'] = pd.to_numeric(t5['t5small_similarity'], errors='coerce')

similarity_sentence_clean_gt = t5.groupby(['grid']).agg({ 
                                            "t5small_similarity": ['min', 'max', 'std']
                                            })

similarity_sentence_clean_gt.sort_values(by='grid', ascending=True, inplace=True)
similarity_sentence_clean_gt.head()

In [ ]:
final = pd.merge(similarity_sentence_clean_gt, clinical, on='grid', how='left')
final.head()

### mean

In [ ]:
df_sent = pd.read_csv(dataout + 'sentence_clean.csv')
df_sent['t5small_similarity_sum'] = pd.to_numeric(df_sent['t5small_similarity'], errors='coerce')


In [ ]:
df_sent['sentence_count'] = ''
buglst = {}
s = 0
for i in df_sent.index:
  try:
    if df_sent['task'][i] == df_sent['task'][i+1]:
      s += 1
    else: 
      df_sent['sentence_count'][i] = s + 1
      s = 0
  except:
    buglst[df_sent['Unnamed: 0'][i]] = df_sent['content'][i]

df_sent.head(20)